# Real-Time Face Mask Detection with Python

In the age of Covid-19, face masks and social distancing have become the new normal. In this article, I will introduce you to a machine learning project on real-time face mask detection with Python.

Installing required libraries 

In [3]:
pip install --upgrade pip

  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tf-nightly

  Created wheel for wrapt: filename=wrapt-1.12.1-py3-none-any.whl size=19557 sha256=ad885c345ede9639239b1351dc67894eb348521892e47fd081da59eba8dcce74
  Stored in directory: c:\users\kishore\appdata\local\pip\cache\wheels\62\76\4c\aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: six
    Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0
  Attempting uninstall: wheel
    Found existing installation: wheel 0.33.6
    Uninstalling wheel-0.33.6:
      Successfully uninstalled wheel-0.33.6
  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.5
    Uninstalling numpy-1.16.5:
      Successfully uninstalled numpy-1.16.5
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\kishore\\anaconda3\\lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
cnn = Sequential([Conv2D(filters=100, kernel_size=(3,3), 
                    activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Conv2D(filters=100, kernel_size=(3,3), 
                    activation='relu'),
                    MaxPooling2D(pool_size=(2,2)),
                    Flatten(),
                    Dropout(0.5),
                    Dense(50),
                    Dense(35),
                    Dense(2)])
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


In [ ]:
import cv2
import numpy as np
labels_dict={0:'No mask',1:'Mask'}
color_dict={0:(0,0,255),1:(0,255,0)}
imgsize = 4 #set image resize
camera = cv2.VideoCapture(0) # Turn on camera
# Identify frontal face
classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
while True:
    (rval, im) = camera.read()
    im=cv2.flip(im,1,1) #mirrow the image
    imgs = cv2.resize(im, (im.shape[1] // imgsize, im.shape[0] //
    imgsize))
    face_rec = classifier.detectMultiScale(imgs) 
    for i in face_rec: # Overlay rectangle on face
        (x, y, l, w) = [v * imgsize for v in i] 
        face_img = im[y:y+w, x:x+l]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=cnn.predict(reshaped)
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(im,(x,y),(x+l,y+w),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+l,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-
        10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',im)
    key = cv2.waitKey(10)
    # stop loop by ESC
    if key == 27: # The Esc key
        break
webcam.release()
cv2.destroyAllWindows()